# Clean - Clean Dataset
This example illustrate how to perform meta-blocking on a clean-clean dataset (data linkage), so when we have two datasets that do not contain duplicates and we want to discover the duplicates between them. 

In [1]:
import sparker
import random
import pandas

## Load the data
sparkER provides wrappers to load CSV and JSON files.

First, we load the first dataset, and we extract the maximum id. The profiles ids of the second dataset will be assigned starting from this one.

*real_id_field* is the field that contains the identifier of the record.

In [2]:
# Profiles contained in the first dataset
profiles1 = sparker.CSVWrapper.load_profiles('/Final_Project/amazon_google/Amazon.csv',
                                             separator=",", header=True,
                                             real_id_field = "id")
# Max profile id in the first dataset, used to separate the profiles in the next phases
separator_id = profiles1.map(lambda profile: profile.profile_id).max()
# Separators, used during blocking to understand from which dataset a profile belongs. It is an array because sparkER
# could work with multiple datasets
separator_ids = [separator_id]

2022-04-01 17:41:54,800 WARN util.Utils: Your hostname, aditi-OMEN-by-HP-Laptop-16-b0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
2022-04-01 17:41:54,801 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-01 17:41:55,121 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-01 17:41:55,572 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
profiles1.take(2)
profiles1.toDF().show(1363)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{title, clickart...| b000jz4hqo|         0|        0|
|[{title, ca inter...| b0006zf55o|         1|        0|
|[{title, noah's a...| b00004tkvy|         2|        0|
|[{title, peachtre...| b000g80lqo|         3|        0|
|[{title, singing ...| b0006se5bq|         4|        0|
|[{title, emc retr...| b000ehpzv8|         5|        0|
|[{title, adobe af...| b00021xhzw|         6|        0|
|[{title, acad upg...| b000gzwjgc|         7|        0|
|[{title, mia's ma...| b0000dbykm|         8|        0|
|[{title, disney's...| b00029bqa2|         9|        0|
|[{title, paper ar...| b0007prnjo|        10|        0|
|[{title, nfs maes...| b000aazr5i|        11|        0|
|[{title, microsof...| b000bhl1r8|        12|        0|
|[{title, spy swee...| b000i82j80|        13|        0|
|[{title, domino d...| b00006hmwc|        14|   

Let's visualize a profile to check if they are correctly loaded

In [4]:
print(profiles1.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'title', 'value': 'clickart 950 000 - premier image pack (dvd-rom)'}, {'key': 'manufacturer', 'value': 'broderbund'}, {'key': 'price', 'value': '0'}], 'original_id': 'b000jz4hqo', 'source_id': 0}


Loads the second dataset and extract the max id (it will be used later)

In [5]:
profiles2 = sparker.CSVWrapper.load_profiles('/Final_Project/amazon_google/GoogleProducts.csv', 
                                              start_id_from = separator_id+1, 
                                              real_id_field = "id")
# Max profile id
max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()

In [6]:
profiles2.toDF().show(500)

+--------------------+--------------------+----------+---------+
|          attributes|         original_id|profile_id|source_id|
+--------------------+--------------------+----------+---------+
|[{name, learning ...|http://www.google...|      1363|        0|
|[{name, superstar...|http://www.google...|      1364|        0|
|[{name, qb pos 6....|http://www.google...|      1365|        0|
|[{name, math miss...|http://www.google...|      1366|        0|
|[{name, productio...|http://www.google...|      1367|        0|
|[{name, video stu...|http://www.google...|      1368|        0|
|[{name, edius pro...|http://www.google...|      1369|        0|
|[{name, qb pos 6....|http://www.google...|      1370|        0|
|[{name, qb credit...|http://www.google...|      1371|        0|
|[{name, vegas mov...|http://www.google...|      1372|        0|
|[{name, qb pos 6....|http://www.google...|      1373|        0|
|[{name, ares risi...|http://www.google...|      1374|        0|
|[{name, evergirl:...|htt

Finally, concatenate the two RDDs

In [7]:
profiles = profiles1.union(profiles2)

In [8]:
profiles.toDF().show(1400)

+--------------------+--------------------+----------+---------+
|          attributes|         original_id|profile_id|source_id|
+--------------------+--------------------+----------+---------+
|[{title, clickart...|          b000jz4hqo|         0|        0|
|[{title, ca inter...|          b0006zf55o|         1|        0|
|[{title, noah's a...|          b00004tkvy|         2|        0|
|[{title, peachtre...|          b000g80lqo|         3|        0|
|[{title, singing ...|          b0006se5bq|         4|        0|
|[{title, emc retr...|          b000ehpzv8|         5|        0|
|[{title, adobe af...|          b00021xhzw|         6|        0|
|[{title, acad upg...|          b000gzwjgc|         7|        0|
|[{title, mia's ma...|          b0000dbykm|         8|        0|
|[{title, disney's...|          b00029bqa2|         9|        0|
|[{title, paper ar...|          b0007prnjo|        10|        0|
|[{title, nfs maes...|          b000aazr5i|        11|        0|
|[{title, microsof...|   

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [9]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
gt = sparker.CSVWrapper.load_groundtruth('/Final_Project/amazon_google/Amzon_GoogleProducts_perfectMapping.csv',
                                         'idAmazon', 'idGoogleBase')

In [10]:
gt.toDF().show(1400)

+---------------+--------------------+
|first_entity_id|    second_entity_id|
+---------------+--------------------+
|     b000jz4hqo|http://www.google...|
|     b00004tkvy|http://www.google...|
|     b000g80lqo|http://www.google...|
|     b0006se5bq|http://www.google...|
|     b00021xhzw|http://www.google...|
|     b0000dbykm|http://www.google...|
|     b0000dbykm|http://www.google...|
|     b00029bqa2|http://www.google...|
|     b0007prnjo|http://www.google...|
|     b000i82j80|http://www.google...|
|     b000i82j80|http://www.google...|
|     b000in6u62|http://www.google...|
|     b00006hvvo|http://www.google...|
|     b00006hvvo|http://www.google...|
|     b0000ycfcw|http://www.google...|
|     b00066dd5m|http://www.google...|
|     b00002sac9|http://www.google...|
|     b000bcz8ng|http://www.google...|
|     b000fm18vi|http://www.google...|
|     b00009apna|http://www.google...|
|     b0009rgzgm|http://www.google...|
|     b000ap2wyw|http://www.google...|
|     b000o24l3q|http://w

In [11]:
# Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles1, profiles2)

In [12]:
len(new_gt)

1300

In [13]:
# We can explore some pairs
random.sample(new_gt, 2)

/tmp/ipykernel_10964/3362377442.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(new_gt, 2)


[(1259, 1819), (641, 1908)]

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [14]:
blocks = sparker.Blocking.create_blocks(profiles, separator_ids, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 14738


In [15]:
blocks.take(500)

[{'block_id': 0, 'profiles': [{0, 512, 514, 1025, 519, 521, 523, 12, 524, 1038, 527, 537, 26, 1051, 538, 27, 540, 542, 33, 545, 35, 546, 550, 39, 40, 1068, 557, 569, 1084, 61, 63, 1092, 583, 584, 585, 591, 592, 1104, 1110, 1111, 600, 1113, 1118, 612, 616, 1130, 107, 620, 1132, 1136, 625, 626, 115, 632, 124, 1150, 638, 643, 1159, 1160, 649, 1162, 653, 655, 1169, 1171, 661, 663, 157, 158, 1185, 1187, 676, 679, 1194, 686, 1204, 1207, 1216, 1217, 1221, 713, 216, 729, 733, 222, 736, 738, 233, 1257, 1259, 1261, 755, 756, 1267, 1268, 1270, 762, 765, 766, 767, 1278, 769, 1282, 772, 776, 266, 267, 783, 1300, 281, 282, 796, 1308, 287, 289, 801, 802, 803, 1325, 816, 309, 826, 828, 322, 834, 840, 1357, 334, 856, 346, 860, 352, 353, 357, 872, 364, 369, 883, 886, 892, 382, 897, 392, 904, 910, 404, 411, 923, 413, 935, 952, 443, 961, 452, 965, 968, 972, 974, 464, 979, 984, 990, 484, 493, 1009, 1010}, {2049, 1539, 2056, 1545, 2058, 2571, 1550, 2069, 3094, 1531, 3608, 1560, 1563, 1564, 3615, 4129, 4133,

Let's continue by using the standard token blocking

In [16]:
blocks = sparker.Blocking.create_blocks(profiles, separator_ids)
print("Number of blocks",blocks.count())

Number of blocks 6834


In [17]:
blocks.take(1)

[{'block_id': 0, 'profiles': [{0, 512, 130, 641, 771, 900, 1285, 135, 392, 1288, 255, 511, 14, 142, 531, 787, 1175, 154, 797, 1183, 544, 293, 549, 423, 1191, 811, 685, 1326, 175, 431, 690, 437, 566, 949, 1333, 706, 580, 70, 582, 713, 460, 590, 591, 338, 211, 212, 600, 857, 218, 987, 732, 860, 223, 480, 865, 248, 871, 492, 748, 1008, 246, 1271, 632, 502, 251, 763, 767}, {3077, 3594, 2571, 3599, 3089, 2578, 1562, 2089, 3119, 1591, 3148, 1615, 1628, 2141, 2654, 2652, 2669, 3697, 2162, 2676, 1659, 2188, 1679, 1696, 2722, 3241, 4279, 1720, 1724, 3260, 1728, 1731, 1736, 2249, 1741, 2769, 2266, 3811, 4324, 4325, 4334, 2289, 3315, 4341, 3337, 4362, 1808, 3350, 1820, 1823, 2847, 2849, 3875, 3876, 4399, 2356, 2359, 3897, 2878, 1860, 2374, 4422, 3917, 4434, 1878, 3930, 2908, 2912, 2914, 1382, 1392, 3957, 4472, 1402, 2946, 1930, 2954, 1421, 3478, 1955, 1445, 1962, 4523, 1453, 3503, 1463, 4539, 3005, 2495, 3521, 3014, 1992, 3020, 2510, 2512, 3539, 2518, 4054, 2020, 1510, 2540, 3574, 3066, 3070}], '

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [18]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [19]:
blocks_purged.take(500)

[{'block_id': 0, 'profiles': [{0, 512, 130, 641, 771, 900, 1285, 135, 392, 1288, 255, 511, 14, 142, 531, 787, 1175, 154, 797, 1183, 544, 293, 549, 423, 1191, 811, 685, 1326, 175, 431, 690, 437, 566, 949, 1333, 706, 580, 70, 582, 713, 460, 590, 591, 338, 211, 212, 600, 857, 218, 987, 732, 860, 223, 480, 865, 248, 871, 492, 748, 1008, 246, 1271, 632, 502, 251, 763, 767}, {3077, 3594, 2571, 3599, 3089, 2578, 1562, 2089, 3119, 1591, 3148, 1615, 1628, 2141, 2654, 2652, 2669, 3697, 2162, 2676, 1659, 2188, 1679, 1696, 2722, 3241, 4279, 1720, 1724, 3260, 1728, 1731, 1736, 2249, 1741, 2769, 2266, 3811, 4324, 4325, 4334, 2289, 3315, 4341, 3337, 4362, 1808, 3350, 1820, 1823, 2847, 2849, 3875, 3876, 4399, 2356, 2359, 3897, 2878, 1860, 2374, 4422, 3917, 4434, 1878, 3930, 2908, 2912, 2914, 1382, 1392, 3957, 4472, 1402, 2946, 1930, 2954, 1421, 3478, 1955, 1445, 1962, 4523, 1453, 3503, 1463, 4539, 3005, 2495, 3521, 3014, 1992, 3020, 2510, 2512, 3539, 2518, 4054, 2020, 1510, 2540, 3574, 3066, 3070}], '

In [20]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8, separator_ids)

In [21]:
blocks_purged.take(500)

[{'block_id': 0, 'profiles': [{0, 512, 130, 641, 771, 900, 1285, 135, 392, 1288, 255, 511, 14, 142, 531, 787, 1175, 154, 797, 1183, 544, 293, 549, 423, 1191, 811, 685, 1326, 175, 431, 690, 437, 566, 949, 1333, 706, 580, 70, 582, 713, 460, 590, 591, 338, 211, 212, 600, 857, 218, 987, 732, 860, 223, 480, 865, 248, 871, 492, 748, 1008, 246, 1271, 632, 502, 251, 763, 767}, {3077, 3594, 2571, 3599, 3089, 2578, 1562, 2089, 3119, 1591, 3148, 1615, 1628, 2141, 2654, 2652, 2669, 3697, 2162, 2676, 1659, 2188, 1679, 1696, 2722, 3241, 4279, 1720, 1724, 3260, 1728, 1731, 1736, 2249, 1741, 2769, 2266, 3811, 4324, 4325, 4334, 2289, 3315, 4341, 3337, 4362, 1808, 3350, 1820, 1823, 2847, 2849, 3875, 3876, 4399, 2356, 2359, 3897, 2878, 1860, 2374, 4422, 3917, 4434, 1878, 3930, 2908, 2912, 2914, 1382, 1392, 3957, 4472, 1402, 2946, 1930, 2954, 1421, 3478, 1955, 1445, 1962, 4523, 1453, 3503, 1463, 4539, 3005, 2495, 3521, 3014, 1992, 3020, 2510, 2512, 3539, 2518, 4054, 2020, 1510, 2540, 3574, 3066, 3070}], '

In [22]:
print("Number of blocks_purged",blocks_purged.count())

Number of blocks_purged 6818


If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [23]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt, separator_ids)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

Recall 0.9984615384615385
Precision 0.0009325663968594497
Number of comparisons 1391858


## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [24]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [25]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [105]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.9923076923076923
Precision 0.002391027530179772
Number of comparisons 539517


The number of edges
The number of matches (only if the groundtruth is provided)
The retained edges

In [106]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|         12|            1|[{0, 1640, 2}, {0...|
|       1914|            1|[{512, 2116, 4}, ...|
|         21|            1|[{130, 3078, 2}, ...|
|        548|            0|[{900, 1544, 2}, ...|
|        152|            0|[{392, 4354, 5}, ...|
|        175|            1|[{1288, 4098, 2},...|
|         50|            0|[{14, 3401, 2}, {...|
|         44|            0|[{142, 3697, 8}, ...|
|        230|            0|[{154, 3995, 2}, ...|
|        643|            1|[{544, 4008, 4}, ...|
+-----------+-------------+--------------------+
only showing top 10 rows



### Reciprocal Weighted Node Pruning

In [107]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.9776923076923076
Precision 0.0031519377845671604
Number of comparisons 403244


In [108]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          7|            1|[{0, 4297, 2}, {0...|
|       1393|            1|[{512, 2116, 4}, ...|
|         16|            1|[{130, 2517, 3}, ...|
|        488|            0|[{900, 1544, 2}, ...|
|        128|            0|[{392, 4354, 5}, ...|
|        144|            1|[{1288, 4098, 2},...|
|         37|            0|[{14, 3401, 2}, {...|
|         40|            0|[{142, 3697, 8}, ...|
|        179|            0|[{154, 3588, 4}, ...|
|        547|            1|[{544, 4008, 4}, ...|
+-----------+-------------+--------------------+
only showing top 10 rows



### Weighted Edge Pruning

In [109]:
results = sparker.WEP.wep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.9907692307692307
Precision 0.002337080282699618
Number of comparisons 551115


In [110]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|         12|            1|[{0, 3241, 5}, {0...|
|       1974|            1|[{512, 1440, 3}, ...|
|         20|            1|[{130, 3078, 2}, ...|
|        548|            0|[{900, 3584, 2}, ...|
|        150|            0|[{392, 4357, 8}, ...|
|        165|            1|[{1288, 3724, 3},...|
|         50|            0|[{14, 3847, 2}, {...|
|         44|            0|[{142, 3072, 6}, ...|
|        217|            0|[{154, 1805, 3}, ...|
|        643|            1|[{544, 3842, 5}, ...|
+-----------+-------------+--------------------+
only showing top 10 rows



### Cardinality Node Pruning

In [37]:
results = sparker.CNP.cnp(
                          blocks_after_filtering,
                          profiles.count(),
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.9246153846153846
Precision 0.009491097161356548
Number of comparisons 126645


In [38]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|         38|            1|[{0, 1634}, {0, 1...|
|        703|            1|[{512, 3367}, {51...|
|         39|            1|[{130, 1504}, {13...|
|         72|            0|[{900, 1389}, {90...|
|         38|            0|[{392, 3058}, {39...|
|         55|            1|[{1288, 4582}, {1...|
|         36|            0|[{14, 3337}, {14,...|
|         34|            0|[{142, 3072}, {14...|
|         68|            0|[{154, 3297}, {15...|
|         63|            0|[{544, 1536}, {54...|
+-----------+-------------+--------------------+
only showing top 10 rows



### Reciprocal Cardinality Node Pruning

In [39]:
results = sparker.CNP.cnp(
                          blocks_after_filtering,
                          profiles.count(),
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.ECBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.8084615384615385
Precision 0.033344966528125894
Number of comparisons 31519


In [40]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|         23|            1|[{0, 4297}, {0, 3...|
|         32|            0|[{512, 1738}, {51...|
|         12|            1|[{130, 1504}, {13...|
|         20|            0|[{900, 1431}, {90...|
|         22|            0|[{392, 2048}, {39...|
|         18|            1|[{1288, 4582}, {1...|
|         12|            0|[{14, 3337}, {14,...|
|         17|            0|[{142, 3072}, {14...|
|         32|            0|[{154, 3082}, {15...|
|         24|            0|[{544, 3876}, {54...|
+-----------+-------------+--------------------+
only showing top 10 rows



### Cardinality Edge Pruning

In [33]:
results = sparker.CEP.cep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.7153846153846154
Precision 0.012002632835589742
Number of comparisons 77483


In [34]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(500)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          1|            1|      [{0, 3241, 5}]|
|        861|            1|[{512, 3202, 7}, ...|
|          0|            0|                  []|
|         55|            0|[{900, 2652, 4}, ...|
|         18|            0|[{392, 4325, 26},...|
|         10|            1|[{1288, 1856, 15}...|
|          1|            0|     [{14, 3337, 5}]|
|         10|            0|[{142, 3172, 9}, ...|
|         29|            0|[{154, 2496, 4}, ...|
|         76|            0|[{544, 4134, 5}, ...|
|        386|            1|[{1326, 3046, 4},...|
|          4|            1|[{690, 1615, 6}, ...|
|          8|            1|[{566, 1679, 4}, ...|
|        129|            0|[{706, 2550, 4}, ...|
|        731|            1|[{580, 4306, 6}, ...|
|          0|            0|                  []|
|         13|            1|[{582, 2688, 4}, ...|
|          2|       

In [124]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(truncate=False)

+-----------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [41]:
edges = results.flatMap(lambda x: x[2])

edges.count()

31519

In [42]:
edges.toDF().show(1400)

+----+----+
|  _1|  _2|
+----+----+
|   0|4297|
|   0|3418|
|   0|3387|
|   0|3350|
|   0|3897|
|   0|2912|
|   0|1659|
|   0|2954|
|   0|3241|
|   0|1741|
|   0|2510|
|   0|2512|
|   0|2479|
|   0|1854|
|   0|4418|
|   0|3913|
|   0|2894|
|   0|3716|
|   0|1704|
|   0|2262|
|   0|2288|
|   0|3335|
|   0|2351|
| 512|2430|
| 512|2442|
| 512|3792|
| 512|2532|
| 512|2518|
| 512|3315|
| 512|4574|
| 512|4456|
| 512|2487|
| 512|2224|
| 512|2232|
| 512|3053|
| 512|3951|
| 512|2034|
| 512|3297|
| 512|3852|
| 512|2401|
| 512|2070|
| 512|1746|
| 512|3234|
| 512|3628|
| 512|1738|
| 512|1649|
| 512|2702|
| 512|2641|
| 512|2644|
| 512|2648|
| 512|2780|
| 512|2657|
| 512|2732|
| 512|2612|
| 512|2457|
| 130|3897|
| 130|3070|
| 130|4437|
| 130|2517|
| 130|3344|
| 130|3157|
| 130|3773|
| 130|2875|
| 130|2638|
| 130|3543|
| 130|3050|
| 130|1504|
| 900|2376|
| 900|1616|
| 900|3022|
| 900|4434|
| 900|2652|
| 900|4399|
| 900|4431|
| 900|3459|
| 900|3373|
| 900|4332|
| 900|4078|
| 900|1930|
| 900|3503|
| 90